In [1]:
!pip install pysqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.4-cp310-cp310-linux_x86_64.whl size=151315 sha256=7f03346bac24c7245f9db11e8875685d9ca6248499c3531afe45aab09f4b1a9c
  Stored in directory: /root/.cache/pip/wheels/04/55/74/e3b53aeb9ff9e933626ee3938ccad69123828115ae34ce028e
Successfully built pysqlite3


In [2]:
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version_info)

(3, 37, 2)


In [3]:
import pandas as pd
from IPython.display import display, HTML

In [4]:
dbname = "music_streaming_1.db"

def printSqlResults(cursor, tbl):
  df=pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
  display(HTML("<b><font color=Green>"+tbl+"</font></b>"+df.to_html(index=False)))

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor=conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor,caption)
  conn.close()

In [5]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")

conn.commit()
conn.close()

In [6]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor=conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

#Insert data into the table

cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")
# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


In [7]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
runSql('Classic Query', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [8]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
runSql('Classic Query with songs like Ye..', qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [9]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
runSql('All genres', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [10]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

runSql('Distinct Genres', qry_distinct)

genre
Pop
Rock
Classic
None


In [11]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [12]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_artist_count)


artist,genre,num_songs
Beatles,Classic,3
DJ,None,1
Ed Sheeran,Rock,3
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [13]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
runSql('One Large Table',qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


In [14]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
runSql('Highly Rated Songs',qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
runSql('Highly Rated Songs',qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


In [15]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id)
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist
ORDER BY COUNT(Listens.song_id) DESC;
"""

runSql('Popular Songs', qry_pop_songs)

song_id,title,artist,count(Listens.song_id)
2,Willow,Taylor Swift,3
1,Evermore,Taylor Swift,2
6,Yesterday,Beatles,2
8,Hey Jude,Beatles,1
7,Yellow Submarine,Beatles,1


In [16]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
runSql('Songs by Ed Sheeran and Taylor Swift', qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
runSql('Songs on both Pop and Rock', qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Bad Blood,Taylor Swift
Evermore,Taylor Swift
Photograph,Ed Sheeran
Shape of You,Ed Sheeran
Shivers,Ed Sheeran
Willow,Taylor Swift


In [17]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
runSql('Songs listened by user_id=1',qry_listen_usr)

title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Yesterday,Beatles


In [18]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""
runSql('Songs shared', qry_sharing)

user_id,song_id
2,1
2,6


In [19]:
#ASSIGNMENT-2. To be submitted as a PDF with queries and screenshots.

#1 Insert the above into the recommendations table
#2 Generate the recommendaions for Minnie

#3 Re-do the generation of recommendations now on the basis of listen time
#4 Generate new recommendations
#5 What are the differences with the static method on #2 above

1 Insert the above into the recommendations table

In [21]:
import sqlite3
import pandas as pd
from IPython.core.display import HTML

# Initialize database connection
db_connection = sqlite3.connect("music_streaming_1.db")  # Replace with your database name
db_cursor = db_connection.cursor()

# Function to run SQL queries and display results
def execute_sql(description, query):
    try:
        db_cursor.execute(query)
        if db_cursor.description:  # Check if the query returns a result set
            result_df = pd.DataFrame(db_cursor.fetchall(), columns=[col[0] for col in db_cursor.description])
            display(HTML(f"<b><font color=Green>{description}</font></b>{result_df.to_html(index=False)}"))
        else:
            print(f"{description}: Query executed successfully.")  # For non-result queries
        db_connection.commit()  # Commit changes for transactional consistency
    except Exception as error:
        print(f"Error executing {description}: {error}")


# Drop the existing Recommendations table
drop_table_query = """
DROP TABLE IF EXISTS Recommendations;
"""
execute_sql("Drop Recommendations Table", drop_table_query)

# Recreate the Recommendations table with AUTOINCREMENT for recommendation_id
create_table_query = """
CREATE TABLE IF NOT EXISTS Recommendations (
    recommendation_id INTEGER PRIMARY KEY AUTOINCREMENT,
    recommendation_time TIMESTAMP,
    user_id INTEGER NOT NULL,
    song_id INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (song_id) REFERENCES Songs(song_id)
);
"""
execute_sql("Create Recommendations Table", create_table_query)


# Query to insert recommendations into the Recommendations table
insert_recommendations_query = """
INSERT INTO Recommendations (recommendation_time, user_id, song_id)
SELECT CURRENT_TIMESTAMP, user_id, song_id
FROM (
    WITH song_similarity AS (
        SELECT u1.song_id AS song1, u2.song_id AS song2, COUNT(*) AS common_users
        FROM Listens u1
        JOIN Listens u2
        ON u1.user_id = u2.user_id AND u1.song_id != u2.song_id
        GROUP BY u1.song_id, u2.song_id
        HAVING COUNT(*) > 1
    )
    SELECT DISTINCT L.user_id, song_similarity.song2 AS song_id
    FROM song_similarity
    JOIN Listens L
    ON L.song_id = song_similarity.song1
    WHERE song_similarity.song2 NOT IN (
        SELECT song_id FROM Listens WHERE Listens.user_id = L.user_id
    )
);
"""

# Execute and insert recommendations
execute_sql("Insert Recommendations into Recommendations Table", insert_recommendations_query)


Drop Recommendations Table: Query executed successfully.
Create Recommendations Table: Query executed successfully.
Insert Recommendations into Recommendations Table: Query executed successfully.


In [26]:
execute_sql("Verify Recommendations", "SELECT * FROM Recommendations;")


recommendation_id,recommendation_time,user_id,song_id
1,2024-12-09 15:40:21,2,1
2,2024-12-09 15:40:21,2,6


In [29]:
# Query to find Minnie's user_id
find_minnie_user_id_query = """
SELECT user_id
FROM Users
WHERE name = 'Minnie';
"""

# Call the function to execute the query
execute_sql("Find Minnie's User ID", find_minnie_user_id_query)

# Query to generate song recommendations for Minnie
generate_minnie_recommendations_query = """
INSERT INTO Recommendations (recommendation_time, user_id, song_id)
SELECT CURRENT_TIMESTAMP, 2, song_id
FROM (
    WITH song_similarity AS (
        SELECT u1.song_id AS song1, u2.song_id AS song2, COUNT(*) AS common_users
        FROM Listens u1
        JOIN Listens u2
        ON u1.user_id = u2.user_id AND u1.song_id != u2.song_id
        GROUP BY u1.song_id, u2.song_id
        HAVING COUNT(*) > 1
    )
    SELECT DISTINCT song_similarity.song2 AS song_id
    FROM song_similarity
    JOIN Listens L
    ON L.song_id = song_similarity.song1
    WHERE L.user_id = 2
    AND song_similarity.song2 NOT IN (
        SELECT song_id FROM Listens WHERE user_id = 2
    )
);
"""

# Function to execute the query for generating recommendations
execute_sql("Generate Song Recommendations for Minnie", generate_minnie_recommendations_query)
execute_sql("Verify Recommendations for Minnie", f"""
SELECT *
FROM Recommendations
WHERE user_id = 2;
""")


user_id
2


Generate Song Recommendations for Minnie: Query executed successfully.


recommendation_id,recommendation_time,user_id,song_id
1,2024-12-09 15:40:21,2,1
2,2024-12-09 15:40:21,2,6
3,2024-12-09 15:52:21,2,1
4,2024-12-09 15:52:21,2,6
5,2024-12-09 15:52:55,2,1
6,2024-12-09 15:52:55,2,6


Task #3: Re-do the Generation of Recommendations Based on Listen Time

In [30]:
# Define the query to generate recommendations for Minnie based on listen time
rec_for_minni = f"""
INSERT INTO Recommendations (recommendation_time, user_id, song_id)
SELECT CURRENT_TIMESTAMP, 2, song_id
FROM (
    WITH song_similarity AS (
        SELECT u1.song_id AS song1, u2.song_id AS song2, COUNT(*) AS common_users
        FROM Listens u1
        JOIN Listens u2
        ON u1.user_id = u2.user_id AND u1.song_id != u2.song_id
        GROUP BY u1.song_id, u2.song_id
        HAVING COUNT(*) > 1
    )
    SELECT DISTINCT song_similarity.song2 AS song_id
    FROM song_similarity
    JOIN Listens L
    ON L.song_id = song_similarity.song1
    WHERE L.user_id = 2
    AND song_similarity.song2 NOT IN (
        SELECT song_id
        FROM Listens
        WHERE user_id = 2
    )
    ORDER BY L.listen_time DESC  -- Prioritize based on listen time
);
"""

# Execute the query to generate the recommendations
execute_sql("Generate Recommendations for Minnie Based on Listen Time", rec_for_minni)


Generate Recommendations for Minnie Based on Listen Time: Query executed successfully.


In [33]:
# Verify the recommendations for Minnie based on listen time
veri_mini_recomm = f"""
SELECT *
FROM Recommendations
WHERE user_id = 2;
"""

execute_sql("Verify Recommendations for Minnie Based on Listen Time", veri_mini_recomm)

# Fetch song details for Minnie's recommendations based on listen time
mini_song_details = """
SELECT R.recommendation_id, R.user_id, S.title, S.artist, S.genre
FROM Recommendations R
JOIN Songs S
ON R.song_id = S.song_id
WHERE R.user_id = 2;
"""

execute_sql("Song Details for Minnie's Recommendations with Listen Time", mini_song_details)


recommendation_id,recommendation_time,user_id,song_id
1,2024-12-09 15:40:21,2,1
2,2024-12-09 15:40:21,2,6
3,2024-12-09 15:52:21,2,1
4,2024-12-09 15:52:21,2,6
5,2024-12-09 15:52:55,2,1
6,2024-12-09 15:52:55,2,6
7,2024-12-09 15:54:04,2,1
8,2024-12-09 15:54:04,2,6


recommendation_id,user_id,title,artist,genre
1,2,Evermore,Taylor Swift,Pop
2,2,Yesterday,Beatles,Classic
3,2,Evermore,Taylor Swift,Pop
4,2,Yesterday,Beatles,Classic
5,2,Evermore,Taylor Swift,Pop
6,2,Yesterday,Beatles,Classic
7,2,Evermore,Taylor Swift,Pop
8,2,Yesterday,Beatles,Classic


Task #4: Generate New Recommendations

In [35]:
# New query to generate recommendations for Minnie based on rating and listen time
gen_new_rec = f"""
INSERT INTO Recommendations (recommendation_time, user_id, song_id)
SELECT CURRENT_TIMESTAMP, {2}, S.song_id
FROM Songs S
JOIN Listens L
ON S.song_id = L.song_id
WHERE S.song_id NOT IN (
    SELECT song_id
    FROM Listens
    WHERE user_id = {2}
)
AND (L.rating >= 4.5 OR L.listen_time IS NOT NULL)
GROUP BY S.song_id
ORDER BY L.rating DESC, L.listen_time DESC;
"""

# Execute the query to generate new recommendations for Minnie
execute_sql("Generate New Recommendations for Minnie Based on Rating and Listen Time", gen_new_rec)

execute_sql("Verify New Recommendations for Minnie", f"""
SELECT *
FROM Recommendations
WHERE user_id = {2};
""")

Generate New Recommendations for Minnie Based on Rating and Listen Time: Query executed successfully.


recommendation_id,recommendation_time,user_id,song_id
1,2024-12-09 15:40:21,2,1
2,2024-12-09 15:40:21,2,6
3,2024-12-09 15:52:21,2,1
4,2024-12-09 15:52:21,2,6
5,2024-12-09 15:52:55,2,1
6,2024-12-09 15:52:55,2,6
7,2024-12-09 15:54:04,2,1
8,2024-12-09 15:54:04,2,6
9,2024-12-09 15:56:57,2,1
10,2024-12-09 15:56:57,2,6


In [36]:
execute_sql("Song Details for New Recommendations", """
SELECT R.recommendation_id, R.user_id, S.title, S.artist, S.genre
FROM Recommendations R
JOIN Songs S
ON R.song_id = S.song_id
WHERE R.user_id = 2;
""")

recommendation_id,user_id,title,artist,genre
1,2,Evermore,Taylor Swift,Pop
2,2,Yesterday,Beatles,Classic
3,2,Evermore,Taylor Swift,Pop
4,2,Yesterday,Beatles,Classic
5,2,Evermore,Taylor Swift,Pop
6,2,Yesterday,Beatles,Classic
7,2,Evermore,Taylor Swift,Pop
8,2,Yesterday,Beatles,Classic
9,2,Evermore,Taylor Swift,Pop
10,2,Yesterday,Beatles,Classic


In [37]:
# Genre-based recommendation query for Minnie
grec_query = f"""
SELECT DISTINCT S.song_id, S.title, S.genre
FROM Songs S
JOIN Listens L
ON S.song_id = L.song_id
WHERE S.genre = (
    SELECT genre
    FROM Songs
    JOIN Listens
    ON Songs.song_id = Listens.song_id
    WHERE user_id = {2}
    GROUP BY genre
    ORDER BY COUNT(*) DESC
    LIMIT 1
)
AND S.song_id NOT IN (
    SELECT song_id
    FROM Listens
    WHERE user_id = {2}
);
"""

# Execute the genre-based recommendation query
execute_sql("Genre-Based Recommendations", grec_query)


song_id,title,genre
6,Yesterday,Classic


In [38]:
new_rec_query = f"""
INSERT INTO Recommendations (recommendation_time, user_id, song_id)
SELECT CURRENT_TIMESTAMP, L.user_id, S.song_id
FROM Songs S
JOIN Listens L
ON S.song_id = L.song_id
WHERE S.song_id NOT IN (
    SELECT song_id
    FROM Listens
    WHERE user_id = L.user_id
)
AND (L.rating >= 4.5 OR L.listen_time IS NOT NULL)  -- Prioritize high ratings or recent listens
GROUP BY L.user_id, S.song_id
ORDER BY L.rating DESC, L.listen_time DESC;
"""
runSql("Generate New Recommendations for Minnie", new_rec_query)


Generate New Recommendations for Minnie: Query executed successfully.


In [40]:
execute_sql("Verify New Recommendations", f"""
SELECT *
FROM Recommendations;
""")

recommendation_id,recommendation_time,user_id,song_id
1,2024-12-09 15:40:21,2,1
2,2024-12-09 15:40:21,2,6
3,2024-12-09 15:52:21,2,1
4,2024-12-09 15:52:21,2,6
5,2024-12-09 15:52:55,2,1
6,2024-12-09 15:52:55,2,6
7,2024-12-09 15:54:04,2,1
8,2024-12-09 15:54:04,2,6
9,2024-12-09 15:56:57,2,1
10,2024-12-09 15:56:57,2,6


In [41]:
execute_sql("Verify New Recommendations", f"""
SELECT R.recommendation_id, S.title, S.artist, S.genre
FROM Recommendations R
JOIN Songs S
ON R.song_id = S.song_id;
""")

recommendation_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Yesterday,Beatles,Classic
3,Evermore,Taylor Swift,Pop
4,Yesterday,Beatles,Classic
5,Evermore,Taylor Swift,Pop
6,Yesterday,Beatles,Classic
7,Evermore,Taylor Swift,Pop
8,Yesterday,Beatles,Classic
9,Evermore,Taylor Swift,Pop
10,Yesterday,Beatles,Classic


Task #5: Differences Between Static and Listen Time/Rating-Based Recommendations

In [ ]:
dy_rec_wsong_nm = """
WITH song_popularity AS (
    SELECT song_id, AVG(listen_time) AS avg_listen_time, COUNT(user_id) AS total_listens
    FROM Listens
    GROUP BY song_id
    HAVING COUNT(user_id) > 1  -- Songs listened to by more than 1 user
),
user_recommendations AS (
    SELECT DISTINCT u.user_id, sp.song_id, s.title AS song_name
    FROM Users u
    CROSS JOIN song_popularity sp
    JOIN Songs s ON sp.song_id = s.song_id
    WHERE sp.song_id NOT IN (
        SELECT song_id
        FROM Listens
        WHERE user_id = u.user_id
    )
    ORDER BY sp.avg_listen_time DESC, sp.total_listens DESC
)
SELECT user_id, song_id, song_name FROM user_recommendations;
"""
runSql("Dynamic Recommendations with Song Names", dy_rec_wsong_nm)